# Baseline Method Applied to Latvia Regions from GEE Dataset

In [3]:
import ee 
import geemap

## Javascript Code before Python Conversion

In [2]:
js_snippet = """
var roi1 = 
    /* color: #d6d6d6 */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.100315625282718, 56.959147246574034],
          [24.100315625282718, 56.9585096175939],
          [24.101847166630222, 56.9585096175939],
          [24.101847166630222, 56.959147246574034]]], null, false),
    roi2 = 
    /* color: #ffffff */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.098904982152252, 56.9528035391015],
          [24.098904982152252, 56.95204585890632],
          [24.10048748547104, 56.95204585890632],
          [24.10048748547104, 56.9528035391015]]], null, false),
    roi3 = 
    /* color: #0b4a8b */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.106323820775962, 56.94755128158278],
          [24.106323820775962, 56.94669401566367],
          [24.108072621053672, 56.94669401566367],
          [24.108072621053672, 56.94755128158278]]], null, false),
    roi4 = 
    /* color: #ffffff */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.11239594083389, 56.945784458271866],
          [24.11239594083389, 56.94503541336855],
          [24.114193020873866, 56.94503541336855],
          [24.114193020873866, 56.945784458271866]]], null, false),
    roi5 = 
    /* color: #ffffff */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.105957872241728, 56.945280987042224],
          [24.105957872241728, 56.944409038748844],
          [24.10787833389639, 56.944409038748844],
          [24.10787833389639, 56.945280987042224]]], null, false),
    roi6 = 
    /* color: #c2c2c2 */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.110208245809112, 56.9448754124318],
          [24.110208245809112, 56.94428581905262],
          [24.11152252822641, 56.94428581905262],
          [24.11152252822641, 56.9448754124318]]], null, false),
    roi7 = 
    /* color: #ffffff */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.120143065990273, 56.94414174756404],
          [24.120143065990273, 56.94319661944267],
          [24.121875773013894, 56.94319661944267],
          [24.121875773013894, 56.94414174756404]]], null, false),
    roi8 = 
    /* color: #ffffff */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.119609693206254, 56.94500148458476],
          [24.119609693206254, 56.94415879031489],
          [24.121251205123368, 56.94415879031489],
          [24.121251205123368, 56.94500148458476]]], null, false),
    roi9 = 
    /* color: #ffffff */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.11625516794117, 56.96225125455494],
          [24.11625516794117, 56.96135923022376],
          [24.11795032403858, 56.96135923022376],
          [24.11795032403858, 56.96225125455494]]], null, false),
    roi10 = 
    /* color: #999999 */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[24.129710662024706, 56.96372685560747],
          [24.129710662024706, 56.96281146988076],
          [24.131722318785876, 56.96281146988076],
          [24.131722318785876, 56.96372685560747]]], null, false);

// Define your regions of interest (ROIs) here
var regions = [
  roi1,
  roi2,
  roi3,
  roi4,
  roi5,
  roi6,
  roi7,
  roi8,
  roi9,
  roi10
];

// Your visualization palette
var vis = ['#CCFFCC','#99FF99','#66FF66','#33CC33','#009900','#006600','#003300','#000000'];

// Initialize a FeatureCollection to store results
var results = ee.FeatureCollection([]);

// Loop over each ROI to calculate the tree count estimate
regions.forEach(function(region, index) {
  var roi = region;

  // Clip the GFCC image to the region
  var canopy_cover = GFCC30TC.filterDate('2015-01-01', '2015-12-31').mosaic().clip(roi);


  // Calculate area of ROI
  var area_of_roi = roi.area(1).getInfo();  

  // Calculate the canopy cover percentage
  var canopy_cover_percentage = canopy_cover.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: roi,
    scale: 30,
    maxPixels: 1e9
  });

  canopy_cover_percentage = canopy_cover_percentage.getInfo();
  
  // Calculate canopy cover proportion
  var canopy_cover_proportion = canopy_cover_percentage['b1'] / 100;
  
  // Estimate the tree count
  var average_tree_area = 60;
  var estimated_tree_count = (area_of_roi / average_tree_area) * canopy_cover_proportion;
  
  // Print the results
  print('Region ' + (index + 1) + ':');
  print('Area of ROI (square meters):', area_of_roi);
  print('Canopy Cover Proportion:', canopy_cover_proportion);
  print('Estimated Tree Count:', estimated_tree_count);

  // Create a feature for this region
  var result_feature = ee.Feature(region, {
    'region_id': 'Region_' + (index + 1),
    'area_of_roi_sqm': area_of_roi,
    'canopy_cover_proportion': canopy_cover_proportion,
    'estimated_tree_count': estimated_tree_count
  });

  // Add the result feature to the collection
  results = results.merge(ee.FeatureCollection([result_feature]));
});

// Export the results as a CSV file
Export.table.toDrive({
  collection: results,
  description: 'Estimated_Tree_Counts_Per_Region',
  fileFormat: 'CSV'
});


"""

In [4]:
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [6]:


# Initialize the Earth Engine API.
ee.Initialize()

# Create the map object
m = geemap.Map()

# Define your regions of interest (ROIs)
roi1 = ee.Geometry.Polygon(
    [[[24.100315625282718, 56.959147246574034],
      [24.100315625282718, 56.9585096175939],
      [24.101847166630222, 56.9585096175939],
      [24.101847166630222, 56.959147246574034]]], None, False)

roi2 = ee.Geometry.Polygon(
    [[[24.098904982152252, 56.9528035391015],
      [24.098904982152252, 56.95204585890632],
      [24.10048748547104, 56.95204585890632],
      [24.10048748547104, 56.9528035391015]]], None, False)

roi3 = ee.Geometry.Polygon(
    [[[24.106323820775962, 56.94755128158278],
      [24.106323820775962, 56.94669401566367],
      [24.108072621053672, 56.94669401566367],
      [24.108072621053672, 56.94755128158278]]], None, False)

roi4 = ee.Geometry.Polygon(
    [[[24.11239594083389, 56.945784458271866],
      [24.11239594083389, 56.94503541336855],
      [24.114193020873866, 56.94503541336855],
      [24.114193020873866, 56.945784458271866]]], None, False)

roi5 = ee.Geometry.Polygon(
    [[[24.105957872241728, 56.945280987042224],
      [24.105957872241728, 56.944409038748844],
      [24.10787833389639, 56.944409038748844],
      [24.10787833389639, 56.945280987042224]]], None, False)

roi6 = ee.Geometry.Polygon(
    [[[24.110208245809112, 56.9448754124318],
      [24.110208245809112, 56.94428581905262],
      [24.11152252822641, 56.94428581905262],
      [24.11152252822641, 56.9448754124318]]], None, False)

roi7 = ee.Geometry.Polygon(
    [[[24.120143065990273, 56.94414174756404],
      [24.120143065990273, 56.94319661944267],
      [24.121875773013894, 56.94319661944267],
      [24.121875773013894, 56.94414174756404]]], None, False)

roi8 = ee.Geometry.Polygon(
    [[[24.119609693206254, 56.94500148458476],
      [24.119609693206254, 56.94415879031489],
      [24.121251205123368, 56.94415879031489],
      [24.121251205123368, 56.94500148458476]]], None, False)

roi9 = ee.Geometry.Polygon(
    [[[24.11625516794117, 56.96225125455494],
      [24.11625516794117, 56.96135923022376],
      [24.11795032403858, 56.96135923022376],
      [24.11795032403858, 56.96225125455494]]], None, False)

roi10 = ee.Geometry.Polygon(
    [[[24.129710662024706, 56.96372685560747],
      [24.129710662024706, 56.96281146988076],
      [24.131722318785876, 56.96281146988076],
      [24.131722318785876, 56.96372685560747]]], None, False)

# List of all regions
regions = [roi1, roi2, roi3, roi4, roi5, roi6, roi7, roi8, roi9, roi10]

# Visualization palette
vis = ['#CCFFCC','#99FF99','#66FF66','#33CC33','#009900','#006600','#003300','#000000']

# Initialize a FeatureCollection to store results
results = ee.FeatureCollection([])

# Define the GFCC30TC image collection (change this to your image collection if necessary)
GFCC30TC = ee.ImageCollection("projects/sat-io/open-datasets/GFCC30TC")

# Loop over each ROI and calculate the tree count estimate
for index, roi in enumerate(regions):
    # Clip the GFCC image to the region
    canopy_cover = GFCC30TC.filterDate('2015-01-01', '2015-12-31').mosaic().clip(roi)

    # Calculate the area of ROI
    area_of_roi = roi.area(1).getInfo()

    # Calculate the canopy cover percentage
    canopy_cover_percentage = canopy_cover.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e9
    })

    canopy_cover_percentage = canopy_cover_percentage.getInfo()

    # If the band is not 'b1', adjust based on your data
    if 'b1' in canopy_cover_percentage:
        canopy_cover_proportion = canopy_cover_percentage['b1'] / 100
    else:
        # Adjust the band name based on your dataset, e.g., 'tree_canopy_cover'
        canopy_cover_proportion = canopy_cover_percentage.get('tree_canopy_cover', 0) / 100

    # Estimate the tree count
    average_tree_area = 60  # Adjust based on your estimate
    estimated_tree_count = (area_of_roi / average_tree_area) * canopy_cover_proportion

    # Print the results
    print(f'Region {index + 1}:')
    print('Area of ROI (square meters):', area_of_roi)
    print('Canopy Cover Proportion:', canopy_cover_proportion)
    print('Estimated Tree Count:', estimated_tree_count)

    # Create a feature for this region
    result_feature = ee.Feature(roi, {
        'region_id': f'Region_{index + 1}',
        'area_of_roi_sqm': area_of_roi,
        'canopy_cover_proportion': canopy_cover_proportion,
        'estimated_tree_count': estimated_tree_count
    })

    # Add the result feature to the collection
    results = results.merge(ee.FeatureCollection([result_feature]))

# Optionally, export the results as a CSV file to Google Drive
# Export.table.toDrive(**{
#     'collection': results,
#     'description': 'Estimated_Tree_Counts_Per_Region',
#     'fileFormat': 'CSV'
# })

# Display the map (add layers if necessary)
m.addLayerControl()
#m


Region 1:
Area of ROI (square meters): 6583.496942579879
Canopy Cover Proportion: 0.1147945205479452
Estimated Tree Count: 12.595822917538671
Region 2:
Area of ROI (square meters): 8084.721187339684
Canopy Cover Proportion: 0.10288087327954437
Estimated Tree Count: 13.862719599585692
Region 3:
Area of ROI (square meters): 10110.020150720682
Canopy Cover Proportion: 0.0968573057191716
Estimated Tree Count: 16.32048854275564
Region 4:
Area of ROI (square meters): 9078.02705421996
Canopy Cover Proportion: 0.1207786711807025
Estimated Tree Count: 18.273867409185897
Region 5:
Area of ROI (square meters): 11293.25542578769
Canopy Cover Proportion: 0.11475850340136054
Estimated Tree Count: 21.599951519878164
Region 6:
Area of ROI (square meters): 5225.972910655914
Canopy Cover Proportion: 0.10096549192364172
Estimated Tree Count: 8.794048761733336
Region 7:
Area of ROI (square meters): 11044.658576286905
Canopy Cover Proportion: 0.12655777584708944
Estimated Tree Count: 23.296457073422534
Reg